# Setup

## Instructions

This notebook utilizes the Kaggle API. Follow these steps in order to get the necessary credentials to continue:

1. Sign up for a Kaggle account at https://www.kaggle.com.
2. Go to the 'Account' tab of your user profile - ```https://www.kaggle.com/{username}/account```.
3. Select 'Create New API Token' under 'API' section.
    - This will trigger the download of kaggle.json, a file containing your API credentials. 
4. Place this file in the location:
    - ~/.kaggle/kaggle.json (for macOS/unix)
    - C:/Users/username/.kaggle/kaggle.json (for Windows) 
    - You can check the exact location, sans drive, with echo %HOMEPATH%). 
    - You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to:
        - $KAGGLE_CONFIG_DIR/kaggle.json (for macOS/unix)
        - %KAGGLE_CONFIG_DIR%\kaggle.json (for Windows)

Note: this notebook uses functions written in Python to query the Kaggle API. **This code will only work for python 3.7 or later**.

## Additional Information

Documentation Guide:
- Kaggle API ([Kaggle](https://www.kaggle.com/docs/api))
- Kaggle API ([GitHub](https://github.com/Kaggle/kaggle-api)) 

## Workflow overview

<img src="../images/Kaggle_workflow.jpg" width=650 height=650 align="left"/>

## Import libraries

In [ ]:
# Allow system to search parent folder for local imports
import sys
sys.path.append('..')

# Kaggle-specific imports
from kaggle import KaggleApi
from kaggle.rest import ApiException

import pandas as pd # For storing/manipulating command data
import json # Reading back the metadata files
from tqdm import tqdm # Gives status bar on loop completion
import itertools # For efficient looping over queries
import os # Exporting saved results
from collections import OrderedDict
from utils import flatten_nested_df
from flatten_json import flatten

In [ ]:
api = KaggleApi()
api.authenticate()

## Query #1: query API based on search terms and search types

Function `get_all_search_outputs` queries the Kaggle API for all combinations of search terms and search types specified and returns the results as a dictionary of dataframes (one dataframe for each query combination)
- Calls function `get_individual_search_output`

In [ ]:
def get_all_search_outputs(search_terms, search_types, flatten_output=False):
    """Call the Kaggle API for each search term and search type. 
    Results are retured in results['{term}_{type}'] = df.
    
    Parameters
    ----------
    search_terms : list-like
        Collection of search terms to query over.
    search_types : list-like
        Collection of search types to query over.
    flatten_output : boolean, optional (default=False)
        Flag for flattening nested columns of output.
    
    Returns
    -------
    results : OrderedDict
        Dictionary consisting of returned DataFrames from get_search_output for each query.
    """
    
    results = OrderedDict()
    
    for search_term, search_type in itertools.product(search_terms, search_types):
        results[(search_term, search_type)] = get_individual_search_output(search_term, search_type, flatten_output)
        
    return results

Function `get_individual_search_output` queries the Kaggle API with the specified search term (e.g., “machine learning”) and search type (must be either “datasets” or “kernels”)
- Searches across all returned pages
- Calls function `_convert_string_csv_output_to_dataframe` which converts results from API (strings in semi-structured table) to dataframe format
- Result is a dataframe (one dataframe per search term/search type combination)

In [ ]:
def get_individual_search_output(search_term, search_type, flatten_output=False):
    """Calls the Kaggle API with the specified search term and returns the search output results.
    
    Parameters
    ----------
    search_term : str
        Keyword to seach for.
    search_type : str 
        Objects to search over (must be either datasets or kernels).
    flatten_output : boolean, optional (default=False)
        Flag for flattening nested columns of output.
    
    Returns
    -------
    df : DataFrame
        DataFrame containing the output of the search query.
    """
    
    # Make sure our input is valid
    assert isinstance(search_type, str), 'Search term must be a string'
    assert search_type in ('datasets', 'kernels'), 'Search can only be conducted over datasets or kernels'
    
    # Use search type to get relevant API function
    list_queries = getattr(api, f'{search_type}_list')
    
    page_idx = 1
    search_df = pd.DataFrame()
    
    # Pulls the records for a single page of results for the given search term
    output = list_queries(search=search_term, page=page_idx)

    # Continue searching until we no longer receive results
    while output:
        if search_type == 'kernels':
            output = [vars(result) for result in output]
        if flatten_output:
            output = [flatten(result) for result in output]
            
        output_df = pd.DataFrame(output)
        output_df['page'] = page_idx

        search_df = pd.concat([search_df, output_df]).reset_index(drop=True)
        
        # Increments the page count for searching
        page_idx += 1

        # Pulls the records for a single page of results for the given search term
        output = list_queries(search = search_term, page=page_idx)
        
    # Modify columns for metadata merge
    search_df = search_df.rename(columns={'id': 'datasetId', 'ref': 'id'})
    if search_type == 'datasets':
        
        search_df = search_df.drop(columns=['viewCount', 'voteCount'])
    search_df = search_df.convert_dtypes()

    
    return search_df

Function `_convert_string_csv_output_to_dataframe` converts raw results from API (strings in semi-structured table) to dataframe format

In [ ]:
def _convert_string_csv_output_to_dataframe(output):
    """Given a string variable in csv format, returns a Pandas DataFrame.
    
    Parameters
    ----------
    output : str
        CSV-styled string to be converted.
    
    Returns
    -------
    df : DataFrame
        DataFrame consisting of data from 'output' string variable.
    """
    
    # Create DataFrame of results
    output = StringIO(output)
    df = pd.read_csv(output)
    
    return df

#### Run query #1 functions - example

In [ ]:
search_terms = ['"machine learning"']
search_types = ['datasets']

In [ ]:
search_output_dict = get_all_search_outputs(search_terms, search_types, flatten_output=True)

In [ ]:
sample_key = (search_terms[0], search_types[0])
sample_df = search_output_dict[sample_key]

In [ ]:
sample_df

## Query #2: query API for full metadata for hits from initial query

Note: Unable to find a way to store metadata in memory as opposed to saving file, but this workaround appears to be functional.

Function `_retrieve_object_json` uses the path for each object (each dataframe in result #1 ordered dictionary to query API for metadata associated with each object). Loads JSON results as dictionary.

In [ ]:
def _retrieve_object_json(object_path, flatten_output=False):
    """Queries Kaggle for metadata json file & returns the json data as a dictionary.
    
    Parameters
    ----------
    object_path : str 
        Path for the dataset.
    flatten_output : boolean, optional (default=False)
        Flag for flattening nested columns of output.
    
    Returns
    -------
    metadata_dict : dict 
        Dictionary containing json metadata.
    """
    
    # Download the metadata
    try:
        api.dataset_metadata(object_path, path='../data/')
    # Error occurs when there is no metadata to return
    except (TypeError, ApiException) as e:
        if (isinstance(e, ApiException) and 
            e.status != 404 and
            'bigquery' not in e.headers['Turbolinks-Location']):
            raise e
        else:
            return None
    else:
        # Access the metadata and load it in as a dictionary
        with open('../data/dataset-metadata.json') as file:
            json_data = json.load(file)

        if flatten_output:
            json_data = flatten(json_data)

        return json_data

Function `get_query_metadata` extracts metadata associated with each object and formats as dataframe
- Calls function `_retrieve_object_json`
- For each object, appends JSON results from `_retrieve_object_json` to a dataframe (converting to dataframe format)
- Output is single dataframe for each search query (matching each dataframe in result #1 ordered dictionary)

In [ ]:
def get_query_metadata(object_paths, flatten_output=False):
    """Retrieves the metadata for the file/files listed in object_paths.
    
    Parameters
    ----------
    object_paths : str/list-like
        String or list of strings containing the paths for the objects.
    flatten_output : boolean, optional (default=False)
        Flag for flattening nested columns of output.
    
    Returns
    -------
    metadata_df : DataFrame
        DataFrame containing metadata for the requested datasets.
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
        
    # Create DataFrame to store metadata in, using columns found in first query, and then add query info
    metadata_df = pd.DataFrame()
        
    # Pulls metadata information for each dataset found above
    for object_path in tqdm(object_paths[:50]):
        # Download & load the metadata
        json_data = _retrieve_object_json(object_path, flatten_output)

        # Store the metadata into our DataFrame created above
        metadata_df = metadata_df.append(json_data, ignore_index=True)
        
    # Modify dtypes
    metadata_df = metadata_df.convert_dtypes()
        
    return metadata_df

Function `get_all_metadata` uses a `for` loop to put dataframes into an ordered dictionary, matching result #1 ordered dictionary
- Calls function `get_query_metadata`

In [ ]:
def get_all_metadata(search_output_dict, flatten_output=False):
    """Retrieves all of the metadata that relates to the provided DataFrames.
    
    Parameters
    ----------
    search_output_dict : dict
        Dictionary of DataFrames from get_all_search_outputs.
    flatten_output : boolean, optional (default=False)
        Flag for flattening nested columns of output.
      
    Returns
    -------
    metadata_dict : OrderedDict
        OrderedDict of DataFrames with metadata for each query.
        Order matches the order of search_output_dict.
    """

    ## Extract IDs from DataFrame, and returns as list of strings
    metadata_dict = OrderedDict()

    for query, df in search_output_dict.items():
        # Kernels do not have metadata
        if 'kernels' not in query:
            print(f'Retrieving {query} metadata')
            # Create object paths
            _, search_type = query
            object_paths = df.id.values

            metadata_dict[query] = get_query_metadata(object_paths, flatten_output)
        
    return metadata_dict

In [ ]:
metadata_dict = get_all_metadata(search_output_dict, flatten_output=True)

In [ ]:
pd.set_option('max_columns', None)
metadata_dict[('"machine learning"', 'datasets')].head()

## Combine results of query #1 and query #2

Function `merge_search_and_metadata_dicts` merges the output dictionaries from query #1 and query #2 to a single ordered dictionary and (optional) saves the results as a single csv file

In [ ]:
def merge_search_and_metadata_dicts(search_dict, metadata_dict, on=None, left_on=None, right_on=None, save=False):
    """Merges together all of the search and metadata DataFrames by the given 'on' key.
    
    Parameters
    ----------
    search_dict : dict
        Dictionary of search output results.
    metadata_dict : dict
        Dictionary of metadata results.
    on : str/list-like, optional (default=None)
        Column name(s) to merge the two dicts on.
    left_on : str/list-like, optional (default=None)
        Column name(s) to merge the left dict on.
    right_on : str/list-like, optional (default=None)
        Column name(s) to merge the right dict on.
    save : boolean/list-like, optional (default=False)
        Specifies if the output DataFrames should be saved.
        If True: saves to file of format 'data/kaggle/kaggle_{search_term}_{search_type}.csv'.
        If list-like: saves to respective location in list of save locations.
            Must contain enough strings (one per query; len(search_terms) * len(search_types)).
            
    Returns
    -------
    df_dict : OrderedDict 
        OrderedDict containing all of the merged search/metadata dicts.
    """
    
    num_dataframes = len(search_dict)
    
    # Ensure the save variable data is proper
    try:
        if isinstance(save, bool):
            save = [save] * num_dataframes
        assert len(save) == num_dataframes
    except:
        raise ValueError('Incorrect save value(s)')

    # Merge the DataFrames
    df_dict = OrderedDict()
    for query_key, save_loc in zip(search_dict.keys(), save):
        search_df = search_dict[query_key]
        if query_key in metadata_dict:
            # Merge small version of "full" dataframe with "detailed" dataframe
            metadata_df = metadata_dict[query_key]
            df_all = pd.merge(search_df, metadata_df, on=on, left_on=left_on, right_on=right_on, how='outer')
        else:
            df_all = search_df
        
        # Save DataFrame
        if save_loc:
            data_dir = os.path.join('data', 'kaggle')
            if isinstance(save_loc, str):
                output_file = save_loc
            elif isinstance(save_loc, bool):
                # Ensure kaggle directory is already created
                if not os.path.isdir(data_dir):
                    os.path.mkdir(data_dir)
                
                search_term, search_type = query_key
                output_file = f'{search_term}_{search_type}.csv'
            else:
                raise ValueError('Save type must be bool or str')

            search_df.to_csv(os.path.join(data_dir, output_file), index=False)
        
        df_dict[query_key] = df_all
    
    return df_dict

In [ ]:
#run merge function
df_dict = merge_search_and_metadata_dicts(search_output_dict, metadata_dict)

In [ ]:
df_dict[sample_key]

#### Example output

In [ ]:
#results of query #1
output_df = search_output_dict[sample_key]

#results of query #2
metadata_df = metadata_dict[sample_key]

#result of merging datasets into "full" dataframe
full_df = df_dict[sample_key]

In [ ]:
sample_df.head()

In [ ]:
metadata_df.head()

In [ ]:
full_df.head()